# OVMS Inference Template
This file is meant to help you with building an inference application for any. 
It contains snippets that handle:
* connecting to a image feed published RTSP stream
* connecting to OVMS via gRPC
* converting images to the proper format 
* running inference
* displaying results 

## Imports and globals

In [ ]:
import cv2, io, time, imutils, datetime, os, traceback, grpc
from imutils.video import VideoStream
import opencv_jupyter_ui as jcv2
import numpy as np

from tensorflow import make_tensor_proto, make_ndarray
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

FPS = 15
MODEL_NAME = ""
RTSP_URL = "rtsp://localhost:8554/server"
GRPC_URL = "localhost:9000"
INPUT_LAYER_NAME = ""
INPUT_LAYER_FORMAT = "float32"
OUTPUT_LAYER_NAME = ""
BATCH_SIZE = 1
HEIGHT = 200
WIDTH = 200

## Open gRPC channel and RTSP Feed
The gRPC channel is used to talk to the OVMS service.
The frames from your webcam are going to be published to an RTSP stream by the `video-capture` service. 

In [ ]:
channel = grpc.insecure_channel(GRPC_URL)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
print("* GRPC Channel succesfully opened")


try:
    rtsp_stream = VideoStream(RTSP_URL).start()
    print("* RTSP Stream succesfully opened")
except Exception as e:
    print("* RTSP Stream is unavailable, please close any other notebooks that might be accessing the stream first")

## Convert the input frame to the correct format
This is highly dependent on the model you are going to use, but there are three steps you might need to take to convert your image
1. Resize, you can see what size your image needs to be by checking the shape of the input layer in `<model_name>.mapping`
2. Transpose and reshape the image to the correct format. You can check the model page on OMZ for more details
3. Convert the image to the datatype your model accepts

The following steps were taken to convert the image for the `face_detection` demo. Feel free to modify it to fit your purpose.

In [ ]:
def load_image(img):
    img = cv2.resize(img, (WIDTH, HEIGHT))
    img = img.transpose(2,0,1).reshape(1,3,HEIGHT, WIDTH)
    img = img.astype(INPUT_LAYER_FORMAT)
    return img 

## Run inference 
This method connects to OVMS using gRPC, sends the (conveted) image and returns the output tensor in numpy format.

In [ ]:
def do_inference(img):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = MODEL_NAME
    request.inputs[INPUT_LAYER_NAME].CopyFrom(make_tensor_proto(img, shape=(img.shape)))
    result = stub.Predict(request, 10.0)
    return make_ndarray(result.outputs[OUTPUT_LAYER_NAME])

## Program Loop

This loop gets new frames from the RTSP Stream, converts images, sends them to OVMS for inference, and displays the result inside an embedded video-box.

In [ ]:
while True:
    try: 
        frame = rtsp_stream.read()
        try: 
            if frame is not None:
                prepared = load_image(frame)
                output = do_inference(prepared)
                
                ## Here you'll probably do something to the output from the inference server
                ## And here you'll probably draw bounding boxes on top of the frame from the camera

                jcv2.imshow("Prediction Output", frame) # This displays the image inside an embedded video-box
            else: 
                continue 
                
            time.sleep(FPS)
        except Exception as e:
            print("error while running inference", e)
            print(traceback.format_exc())
          
    except Exception as e:
        print("error while grabbing RTSP stream", e)